<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_30WMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-11-21 18:59:17


In [2]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=6eceffbc9a1f31f6d5e023f33a49a9460bd376485eb306d7dead33275e81c47f
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
#mypf = mypf[~mypf['Criteria'].isin(['V40','V40N'])]
mypf = mypf[mypf['TFrame'] == 'W']

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

all_prospects = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

all_prospects.sort()
#stock_ids

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 10
      #mid_window = 50
      long_window = 30
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1wk')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['30_WMA'] = round(stock_df['Close'].ewm(span=long_window, adjust=False).mean(),0)
      stock_df['Dev%_30'] = round((stock_df['Close'] - stock_df['30_WMA'])*100/stock_df['30_WMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):
    #date = (datetime.date.today() + datetime.timedelta(days=0))
    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] > df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] > 60) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['30_WMA']) & (df_prec_dev['RSI_14'] < 60) ), -1.0, df_prec_dev['Signal'])

    df_prec_dev['Position'] = df_prec_dev['Signal'].diff()

    df_pos = df_prec_dev[(df_prec_dev['Position'] >= 1) | (df_prec_dev['Position'] <= -1)]
    df_pos = df_pos[df_pos['Signal'] != 0]
    df_pos['Prev_Signal'] = df_pos['Signal'].shift(1)
    df_pos = df_pos[df_pos['Signal'] != df_pos['Prev_Signal']]
    df_pos.drop('Prev_Signal', axis=1, inplace=True)

    return df_pos.tail(1)

def get_weekly_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Criteria'] = mypf[(mypf['Symbol']) == stock_id.upper()]['Criteria'].values[0]
        df_buy_sell = df_buy_sell.append(df_tmp)


    df_buy_sell['Position'] = df_buy_sell['Position'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()


In [7]:
df_weekly_buy_sell = get_weekly_buy_sell()
df_weekly_buy_sell

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-02-27 00:00:00+05:30,3052.0,2689.0,13.49,POLYCAB.NS,5343.05,2499.71,22.09,80.0,1.0,Buy,V40N
2023-02-27 00:00:00+05:30,543.0,647.0,-16.02,AARTIIND.NS,1099.51,449.40,20.90,27.0,-1.0,Sell,VR
2023-02-27 00:00:00+05:30,1828.0,1991.0,-8.19,AAVAS.NS,3068.50,1353.75,35.03,43.0,-1.0,Sell,VR
2023-02-27 00:00:00+05:30,4497.0,5420.0,-17.03,FINEORG.NS,7073.89,4129.51,8.90,15.0,-1.0,Sell,V40N
2023-02-27 00:00:00+05:30,1853.0,1976.0,-6.25,VINATIORGA.NS,2326.96,1721.80,7.59,34.0,-1.0,Sell,V40N
...,...,...,...,...,...,...,...,...,...,...,...
2023-10-30 00:00:00+05:30,936.0,964.0,-2.93,TANLA.NS,2031.49,521.12,79.57,47.0,-1.0,Sell,AR
2023-10-30 00:00:00+05:30,1320.0,1228.0,7.51,RADICO.NS,1440.30,975.43,35.35,61.0,1.0,Buy,V40N
2023-11-06 00:00:00+05:30,1621.0,1420.0,14.14,METROPOLIS.NS,3398.33,1200.40,35.03,63.0,1.0,Buy,VR


In [8]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Sell'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-10-16 00:00:00+05:30,226.0,237.0,-4.85,BANDHANBNK.NS,612.93,194.47,15.96,44.0,-1.0,Sell,VR
2023-10-16 00:00:00+05:30,219.0,222.0,-1.26,APEX.NS,402.28,190.56,15.03,49.0,-1.0,Sell,AR
2023-10-16 00:00:00+05:30,563.0,574.0,-1.90,SBIN.NS,616.05,495.72,13.59,45.0,-1.0,Sell,VR
2023-10-16 00:00:00+05:30,87.0,88.0,-1.25,SAIL.NS,127.95,73.45,18.31,47.0,-1.0,Sell,VR
2023-10-16 00:00:00+05:30,934.0,937.0,-0.33,FINCABLES.NS,1126.60,506.19,84.51,48.0,-1.0,Sell,V40N
2023-10-16 00:00:00+05:30,392.0,405.0,-3.25,WIPRO.NS,706.55,361.75,8.32,44.0,-1.0,Sell,VR
2023-10-23 00:00:00+05:30,1412.0,1439.0,-1.89,INDIGOPNTS.NS,2623.10,1006.05,40.33,47.0,-1.0,Sell,V40N
2023-10-23 00:00:00+05:30,421.0,426.0,-1.13,SIS.NS,605.18,320.65,31.36,48.0,-1.0,Sell,V40N
2023-10-23 00:00:00+05:30,755.0,805.0,-6.19,ROSSARI.NS,1525.49,584.19,29.27,41.0,-1.0,Sell,VR


In [9]:
df_weekly_buy_sell[df_weekly_buy_sell['Position'] == 'Buy'].tail(20)

,Close,30_WMA,Dev%_30,Stock,Max,Min,Gained%,RSI_14,Signal,Position,Criteria
Date,,,,,,,,,,,
2023-07-10 00:00:00+05:30,485.0,441.0,10.06,SUNTV.NS,669.49,408.13,18.93,66.0,1.0,Buy,V40N
2023-07-17 00:00:00+05:30,339.0,311.0,8.96,VAIBHAVGBL.NS,961.00,270.83,25.13,65.0,1.0,Buy,VR
2023-07-24 00:00:00+05:30,403.0,376.0,7.15,ABSLAMC.NS,663.47,307.22,31.14,64.0,1.0,Buy,AR
2023-07-24 00:00:00+05:30,405.0,369.0,9.80,LICHSGFIN.NS,511.43,314.23,28.94,65.0,1.0,Buy,AR
2023-08-07 00:00:00+05:30,7.0,6.0,24.17,JPPOWER.NS,14.50,5.55,34.23,66.0,1.0,Buy,VR
2023-08-07 00:00:00+05:30,16.0,15.0,10.00,HATHWAY.NS,48.95,12.95,27.41,63.0,1.0,Buy,VR
2023-08-07 00:00:00+05:30,87.0,76.0,14.80,BANKINDIA.NS,109.40,68.23,27.88,62.0,1.0,Buy,VR
2023-08-07 00:00:00+05:30,1648.0,1328.0,24.10,GLAND.NS,4287.20,922.25,78.69,70.0,1.0,Buy,VR
2023-08-14 00:00:00+05:30,569.0,504.0,12.83,WSTCSTPAPR.NS,761.75,459.99,23.62,62.0,1.0,Buy,VR


In [10]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [11]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)